In [1]:
import torch 
from torch import nn

import ray
from ray.rllib.agents import ppo
from ray.rllib.models import ModelCatalog
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.rllib.utils.annotations import override

#from models import VisualEncoder
from train import *
from wrappers_2 import *

/root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [2]:
class VisualEncoder(nn.Module):
    def __init__(self):
        super().__init__()

        self.cnn = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=8, stride=4, padding=0),  
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=0), 
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=0), 
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=0),
            nn.ReLU(), 
            nn.Conv2d(64, 512, kernel_size=2, stride=1, padding=0),
            nn.ReLU(),
            nn.Flatten(),
        )

    def forward(self, x):
        return self.cnn(x)

In [3]:
class MyModelClass(TorchModelV2, nn.Module):
    def __init__(self, obs_space, action_space, num_outputs, model_config, name):
        TorchModelV2.__init__(self, obs_space, action_space, num_outputs, model_config, name)
        nn.Module.__init__(self)
        features_dim = 512
        self.encoder = VisualEncoder()
        self.encoder.load_state_dict(
            torch.load("/IGLU-Minecraft/models/AnnaCNN/encoder_weigths.pth", map_location=torch.device('cpu'))
        )
        self.action_head = nn.Linear(features_dim, action_space.n)
        self.value_head = nn.Linear(features_dim, 1)
        self.last_value = None
        
        self.use_cuda = torch.cuda.is_available()
        if self.use_cuda:
            self.encoder.cuda()
            self.action_head.cuda()
            self.value_head.cuda()
        
    @override(TorchModelV2)
    def forward(self, input_dict, state, seq_lens):
        obs = input_dict['obs'].permute(0, 3, 1, 2).float() / 255.0
        if self.use_cuda:
            obs.cuda()
            
        features = self.encoder(obs)
        action = self.action_head(features)
        self.last_value = self.value_head(features).squeeze(1)
        return action, state
    
    @override(TorchModelV2)
    def value_function(self):
        assert self.last_value is not None, "must call forward() first"
        return self.last_value

In [4]:
ModelCatalog.register_custom_model("my_torch_model", MyModelClass)

In [5]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

def env_creator(env_config):
    env = gym.make('IGLUSilentBuilder-v0', max_steps=1000)
    env.update_taskset(TaskSet(preset=['C32']))
    env = PovOnlyWrapper(env)
    env = SelectAndPlace(env)
    env = Discretization(env, flat_action_space('human-level'))
    return env

from ray.tune.registry import register_env
register_env("my_env", env_creator)

from ray import tune
from ray.rllib.agents.ppo import PPOTrainer

In [6]:
from ray.tune.integration.wandb import WandbLogger

analysis = tune.run(PPOTrainer, 
         config={
             "env": "my_env", 
             "framework": "torch",
             "num_gpus": 1,
             "num_workers": 1,
             "sgd_minibatch_size": 256,
             "clip_param": 0.2,
             "entropy_coeff": 0.01,
             "lambda": 0.95,
             "train_batch_size": 1000,
             "model": {
                    # Specify our custom model from above.
                    "custom_model": "my_torch_model",
                    # Extra kwargs to be passed to your model's c'tor.
                    "custom_model_config": {},
              },
             "logger_config": {
                  "wandb": {
                      "project": "IGLU-Minecraft",
                      "name": "PPO C32 pretrained (AnnaCNN) (3 noops after placement)"
                  }
              }

        },
        loggers=[WandbLogger])

/root/miniconda/envs/py37/lib/python3.7/site-packages/ray/_private/services.py:238: UserWarning: Not all Ray Dashboard dependencies were found. To use the dashboard please install Ray using `pip install ray[default]`. To disable this message, set RAY_DISABLE_IMPORT_WARNING env var to '1'.
  warnings.warn(warning_message)


Trial name,status,loc
PPO_my_env_21374_00000,PENDING,


2021-10-06 14:10:16,231	INFO wandb.py:170 -- Already logged into W&B.
2021-10-06 14:10:16,240	ERROR syncer.py:72 -- Log sync requires rsync to be installed.
wandb: Currently logged in as: linar (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
(pid=175) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=175)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


(pid=175) 2021-10-06 14:10:19,757	INFO ppo.py:159 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
(pid=175) 2021-10-06 14:10:19,757	INFO trainer.py:728 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=176) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=176)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=175) 2021-10-06 14:10:28,630	INFO trainable.py:109 -- Trainable.setup took 11.389 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
(pid=175) 2021-10-06 14:10:28,631	WARNING util.py:55 -- Install gputil for GPU system monitoring.
(pid=176) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: User

Result for PPO_my_env_21374_00000:
  agent_timesteps_total: 1000
  custom_metrics: {}
  date: 2021-10-06_14-11-27
  done: false
  episode_len_mean: 966.0
  episode_media: {}
  episode_reward_max: 0.0
  episode_reward_mean: 0.0
  episode_reward_min: 0.0
  episodes_this_iter: 1
  episodes_total: 1
  experiment_id: 49db762336a445169df010321c0a100e
  hostname: linar-Z390-GAMING-X
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 0.19999999999999996
          cur_lr: 5.000000000000001e-05
          entropy: 0.5071485141913096
          entropy_coeff: 0.009999999999999998
          kl: 0.02036470083990674
          policy_loss: 0.017980292439460754
          total_loss: 0.0417120463318295
          vf_explained_var: 0.6013163924217224
          vf_loss: 0.024730300173784296
    num_agent_steps_sampled: 1000
    num_agent_steps_trained: 1000
    num_steps_sampled: 1000
    num_steps_trained: 1000
  iterations_since_restore: 1
  node_ip: 192.168.3.5
  nu

...

agent_timesteps_total,437000
episode_len_mean,45.24
episode_reward_max,10.0
episode_reward_mean,9.89
episode_reward_min,5.0
episodes_this_iter,25
episodes_total,2903
info/learner/default_policy/learner_stats/cur_kl_coeff,0.00196
info/learner/default_policy/learner_stats/cur_lr,5e-05
info/learner/default_policy/learner_stats/entropy,0.37311
info/learner/default_policy/learner_stats/entropy_coeff,0.01


agent_timesteps_total,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
episode_len_mean,█▇▆▆▆▅▅▅▅▅▅▅▅▅▄▄▄▄▄▅▅▅▅▅▄▄▄▄▂▂▂▁▁▁▁▁▁▁▁▁
episode_reward_max,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆▆▆▆████████████████
episode_reward_mean,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▃▃▃▄▃▅▇▇█████████
episode_reward_min,▆▆▆▆▆▆▆▆▆▆▆▆▆▅▅▅▅▅▅▅▅▅▅▅▅▅▃▁▁▃▆▆▇▆▇▆▇▇▇█
episodes_this_iter,▁▁▁▂▁▁▂▁▁▁▁▂▁▂▂▂▂▁▁▁▁▂▂▂▁▂▂▁▃▇▆▄▇▄▅▅▁▆█▅
episodes_total,▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▅▅▆▆▇▇█
info/learner/default_policy/learner_stats/cur_kl_coeff,▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▃▂▂▂▂▃▃▃▃▃▅▄█▂▂▁▁▁▁▁▁▁▁▁
info/learner/default_policy/learner_stats/cur_lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
info/learner/default_policy/learner_stats/entropy,▄▆█▇▇▇▇█▇▇▇▅▅▇▇▆▆▅▆▅▆▆▅▅▅▅▄▅▄▂▂▄▁▅▂▃▆▂▁▃
info/learner/default_policy/learner_stats/entropy_coeff,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


2021-10-07 03:53:33,867	WARNING util.py:164 -- The `process_trial` operation took 3.628 s, which may be a performance bottleneck.


(pid=176) Attempted to send kill command to minecraft process and failed with exception timed out


TuneError: ('Trials did not complete', [PPO_my_env_21374_00000])